In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import heapq
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
df = pd.DataFrame({'fm':['a','a','b','b','b','c','e','f'],
                   'to':['b','d','a','c','d','d','d','d']})
rt.tile([rt.chordDiagram(df, [('fm','to')], draw_labels=True, txt_h=20),
         rt.chordDiagram(df, [('fm','to')], draw_labels=True, txt_h=20, link_style='wide'),
         rt.chordDiagram(df, [('fm','to')], draw_labels=True, txt_h=20,                    equal_size_nodes=True),
         rt.chordDiagram(df, [('fm','to')], draw_labels=True, txt_h=20, link_style='wide', equal_size_nodes=True)])

In [ ]:
rt.bezierCurve((0,0),(1,1),(2,2),(3,3))(0.9)